# Part 5 - Galaxy Classification Using Convolutional Neural Networks

In today's Day 5 tutorial you will be working through an example in which we will be classifying the morphology (i.e. the physical shape) of galaxies in the Sloan Digital Sky Survey (SDSS) using a convolutional neural network named GaMorNet.

Note: The procedural portion of this tutorial is largely based on the GaMorNet tutorials available as part of the GaMorNet documentation (https://gamornet.readthedocs.io/en/latest/tutorials.html). All mentions of "the paper" in this tutorial, refer to [Ghosh et. al. (2020)](https://iopscience.iop.org/article/10.3847/1538-4357/ab8a47).


## Introduction

Galaxies in the Universe can take on a wide variety of shapes and sizes. Galaxy morphological classification, or classifying galaxies by their shape and visual appearance, is a system used by astronomers to divide galaxies into groups. Early observations of a few hundred nearby, bright galaxies by Edwin Hubble in the 1920s led to the development of what is known as the “Hubble Sequence”, a classification scheme that identifies three main morphological types: spirals, lenticulars, and ellipticals (Hubble 1926; Sandage 1961). Although galaxies can be classified by eye, the advent of large astronomical surveys such as the SDSS has ushered in an era of big datasets and modern efforts to classify galaxies by shape largely use computational methods.

### The Shape of Galaxies in the Universe

<img src="https://github.com/heidiawhite/pasea-arp/blob/main/PythonML_workshop/Part_5/imgs/main_image_deep_field_smacs0723-5mb.jpg?raw=1" alt="Alternative text" />

The above image was recently produced by the James Webb Space Telescope (JWST; NASA). This first Deep Field image features galaxy cluster SMACS 0723, and it is teeming with thousands of galaxies – including the faintest objects ever observed at infrared wavelengths. JWST’s image took roughly 12.5 hrs of telescope time to create and covers a tiny patch of sky (approximately the size of a grain of sand held at arm’s length by someone on the ground) and reveals thousands of galaxies of all shapes and sizes in just a sliver of the vast universe. This image is an excellent example of the wonderfully diverse spectrum of galaxy morphology in the Universe.

### `GaMorNet`

In the tutorial below we will be using The Galaxy Morphology Network (GaMorNet). GaMorNet is a convolutional neural network that can classify galaxies as being "disk-like", "elliptical-like" or "indeterminate" based on how light is distributed within the galaxy. The advantage of GaMorNet is that it doesn’t need a large amount of training data to provide reliable predictions and can work across different types of data sets.

Although GaMorNet is a far more sophisticated neural network than the simple example we developed to classify flowers, the basic principles remain the same. Here, GaMorNet assesses galaxy shapes by using physically relevant property of their **bulge-to-total** luminosity ratio. 

 - Galaxies with light mainly distributed in disk structures are said to be **disk-dominated**. This is the case in, for example, spiral galaxies like our own Milky Way.

- Galaxies with light primarily in large, round bulge structures are said to be **bulge-dominated**. Examples of this include massive elliptical galaxies like Messier 87 (also known as Virgo A or NGC 4486).

- Galaxies where it's not entirely clear which component dominates (or in cases where the algorithm is unable to make a classification are said to be **indeterminate**.

<img src="https://github.com/heidiawhite/pasea-arp/blob/main/PythonML_workshop/Part_5/imgs/galaxyzoo_spiral_elliptical.jpeg?raw=1" alt="Alternative text" />

The above image shows an example of disk vs. elliptical type galaxy.

GaMorNet assesses the distribution of light within each galaxy in an image and reports back a likelihood prediction that the galaxy is bulge-dominated, disk-dominated, or indeterminate. Because of this, each galaxy receives *three* likelihood predictions, which each range between 0 and 1.

### Running `GaMorNet` in Google Colab

Although this tutorial can be run on any machine which has GaMorNet installed, it's pretty handy to run this on Google Colab as you can easily use Colab's GPUs for this tutorial.

Note that with the free version of CoLab, you will only have access to a limited amount of memory. Thus, the number of images we use here for training/testing our network is very small in comparison to how many would be used in astronomy research. In reality, GaMorNet can handle hundreds of thousands of images! 

This first section is meant to be run only when following this tutorial in Google Colab.

### Make things Fast! 

Before we dive in, let's make sure we're using a Google CoLab GPU for this tutorial.

To do this, go to the menus at the top and select "Runtime" -> "Change runtime type" -> "Hardware accelerator" -> "GPU".

### Installing Libraries Needed for this Tutorial

Let's first begin by using some *pip* commands to install some required libraries.

In [ ]:
!pip install matplotlib
!pip install astropy
!pip install numpy

### Install GaMorNet

Next, let's install the GaMorNet software.

Note: This command make take a few minutes to run!

In [ ]:
import tensorflow as tf
tf.version

In [ ]:
# Oct 11, 2022
# tensorflow._api.v1.version works with 2.3
# tensorflow v2 works with 2.2
# Gamornet requirements:
# tensorflow-gpu ~=1.13"
# tflearn ~=0.3"
# keras ~=2.2,<2.4"
!pip install keras==2.2

In [ ]:
import keras
keras.__version__

In [ ]:
import os
# Suppressing TF warnings and info for a cleaner environ
# Set this to 0,1 for info and warnings respectively.
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

!pip install -q --upgrade gamornet

In [ ]:
##Checking which version of Tensorflow & GaMorNet is being used and whether the installation worked.
import gamornet
print(tf.__version__)
print(gamornet.__version__)
from gamornet.keras_module import gamornet_train_keras, gamornet_tl_keras, gamornet_predict_keras
from gamornet.tflearn_module import gamornet_train_tflearn, gamornet_tl_tflearn, gamornet_predict_tflearn


The following snippet will verify that we have access to a GPU.

In [ ]:
#Checking access to GPU
import tensorflow as tf
if tf.test.gpu_device_name() != '/device:GPU:0':
  print('WARNING: GPU device not found.')
else:
  print('SUCCESS: Found GPU: {}'.format(tf.test.gpu_device_name()))

## Training with `GaMorNet`

GaMorNet can quite easily be trained from scratch using images. 

In this demonstration, we will use 90 *simulated* SDSS images for the purposes of training and 10 *simulated* SDSS images for validation. All these simulated images come from the set of simulated galaxies created for the paper referenced above. Note: these are not images of actual galaxies. They have been created using software to mimic the wide variety of galaxies observed in the wild.

All these images contain disk + bulge components. As described in the paper, Ghosh et al. (2020) has also convolved these simulations with a representative point-spread-function and added representative noise to better mimic actual scientific data. 

### Downloading the Data

First, let's download the images that we are going to use to train GaMorNet. We will download these into the local filesystem from Yale Astronomy's FTP service, where these are hosted.

We are going to download all the 100 images (90+10) as a single archive and then export it to a single folder called `training_imgs`. The images are in the FITS format and are named `output_img_xx.fits` where xx runs from 0 to 99.

We are also going to download the `sim_para.txt` file containing the ground-truth parameters for the above galaxies. Using these values, we are going to calculate the bulge-to-total light ratio of each galaxy and determine the labels to be used during the training process. 


*Tip: The `%%bash` command lets Colab know that all the commands in this shell needs to be passed the local unix virtual environment.*

*Tip: To view the files in use on Colab, click the folder icon on the left sidebar.*

In [ ]:
%%bash
#get zip and txt file from server
wget ftp://ftp.astro.yale.edu/pub/aghosh/gamornet_tutorial_files/train_images/training_imgs.tar.gz
wget ftp://ftp.astro.yale.edu/pub/aghosh/gamornet_tutorial_files/train_images/sim_para.txt

#Unzip the Archive
tar -xvf training_imgs.tar.gz

In the cell below, write a code block which visualizes two of the training set FITS files and displays them side by side.

Note: to do this you may want to use `subplots` in `matplotlib`. The documentation on its functionality can be found [here](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.subplots.html).

In [ ]:
#Your Code Here

***

#### Q - Iterate through visualizing some of the training data images. How well do they resemble real galaxies? Why might astronomers choose to use simulated images as training data?

***

Ans. Here

### Preparing the Data

In this section, we will generate the training and validation image arrays as well as the corresponding labels to be used during the networks' training process.


First, let's read in the `.txt` file and calculate the difference in disk and bulge magnitudes (i.e. brightnesses) for each of the galaxies.

In [ ]:
import pylab as plt

#Let's read in the sim_para.txt file 
gal_para = plt.genfromtxt("./sim_para.txt",names=True,usecols = (4,11))
 
#difference b/w the integrated magnitude of the disk and bulge components. 
#The rows in the file and thus the elements in the array below correspond to
#the numbers in the names of the image files. (i.e. the 0th element corresponds
#to output_img_0.fits)
disk_bulge_mag = gal_para["Inte_Mag"] - gal_para["Inte_Mag_2"]

Next, let's define two convenience functions, which will assist us in creating the image and label arrays. The first reshapes each data array to add a third dimension. The second function adds a vector that acts as a label for which type of galaxy the array represents. This label is determined by using the magnitudes calculated in the cell above.

In [ ]:
# Convenience Function to get and return images as numpy arrays

def image_handler(i):
  return np.reshape(fits.getdata("./train_images/output_img_"+str(i)+".fits",
                                 memmap=False),newshape=(167,167,1)) 
  #We use the reshape command just to add the extra 3rd dimension. The image is 
  #originally 167*167. So, in essence no re-sizing is taking place in the X or Y
  #directions.


# Convenience Function to get and return the training labels of each galaxy
# in the one-hot encoding format. i.e. disk-dominated galaxies will be represented
# by the array [1,0,0], bulge-dominated by [0,0,1] and indeterminate by [0,1,0]

def label_handler(i):
  
  target_vect = [0]*3

  if (disk_bulge_mag[i] < -0.22): #  (Lb/LT) < 0.45
    target_vect[0] = 1  #disk-dominated       
  
  elif ( -0.22 <=  disk_bulge_mag[i] <= 0.22):
      target_vect[1] = 1 #indeterminate
  
  else: #  (Lb/LT) > 0.55
      target_vect[2] = 1 #bulge-dominated

  return target_vect

Now, we are going to use the first 90 images to create the training set and the last 10 to create the validation set. We are multi-threading the process below -- although this is an absolute overkill for 100 images, it's very handy while dealing with large numbers of images. 

In [ ]:
from multiprocessing import Pool
import numpy as np
from astropy.io import fits

NUM_THREADS = 2

pl = Pool(NUM_THREADS)
training_imgs = np.array(pl.map(image_handler,range(0,90)))
training_labels = np.array(pl.map(label_handler,range(0,90)))

valdiation_imgs = np.array(pl.map(image_handler,range(90,100)))
validation_labels = np.array(pl.map(label_handler,range(90,100)))

There are two modules available for training GaMorNet:

 - `Keres`
 - `TFLearn`

 These two models function somewhat similarly, and differ primarily in terms of their individual efficacies in classifying bulge and disk galaxies. More info on these two approaches can be explored here: https://gamornet.readthedocs.io/en/latest/usage_guide.html

## Training GaMorNet using Keras

Now, we will be using the images and the labels generated above to train GaMorNet. Explore the documentation for `Keras` to learn about what each of these arguments represent in the `gamornet_train_keras` function:

https://gamornet.readthedocs.io/en/latest/api_docs.html#gamornet.keras_module.gamornet_train_keras

In [ ]:
from gamornet.keras_module import gamornet_train_keras

model = gamornet_train_keras(training_imgs,training_labels,valdiation_imgs,
                             validation_labels,input_shape='SDSS', epochs=50, 
                             checkpoint_freq=25, batch_size=64, lr=0.0001, 
                             loss='categorical_crossentropy')

The above command trains a model using the images we prepared for 50 epochs using a learning rate of 0.0001 and a categorical cross-entropy loss function. The `checkpoint_freq = 25` parameter also ensures that every 25 epochs, a snapshot of the model is saved. These models are named as `model_x.hdf5` where x refers to the epoch at which the model was saved. The `input_shape` parameter specifies the shape of the input images. Setting this to `SDSS` automatically sets the value to `(167,167,1)`

For an explanation of the different input parameters of `gamornet_train_keras`, please have a look at the [API documentation](https://gamornet.readthedocs.io/en/latest/api_docs.html).

In the output above, the `accuracy` and `loss` refer to the metrics calculated on the training set at the end of each epoch while `val_loss` and `val_accuracy` refer to the metrics calculated on the validation data.

It's really important to understand the `loss` and `accuracy` output in the context of machine learning.

 - A **loss function** is used to optimize a machine learning algorithm. The "loss" is calculated on training and validation and its interpretation is based on how well the model is doing in evaluating these two sets. The "loss" is the sum of errors made for each example in training or validation sets. Loss value implies how poorly or well a model behaves after each iteration of optimization.

 - An **accuracy metric** is used to measure the algorithm’s performance in an interpretable way. The accuracy of a model is usually determined after the model parameters and is calculated in the form of a percentage. It is the measure of how accurate your model's prediction is compared to the true data. For example, suppose you have 1000 test samples. If your model is able to classify 990 of them correctly, then the model’s accuracy will be 99.0%.

***

#### Q - Read through the training output from `Keras` above. How do the loss and accuracy values evolve over time as the training proceeds? Provide an interpretation.

***

Ans. Here

Congrats! You have trained your first GaMorNet model!! You can have a look at the model's structure using the command below. 

In [ ]:
model.summary()

**Important:**
The above process also generates a `metrics.csv` file, which contains the loss and accuracy calculated on the validation as well as the training data. 

We highly recommend using the data in this file to check how the loss and accuracies vary with training. This is extremely helpful in judging whether the model was trained properly and sufficiently. 

## Training `GaMorNet` using TFLearn

Now, we will be using the images and the labels generated above to train GaMorNet using `TFLearn`.

Read up on the the documentation for this module [here](https://gamornet.readthedocs.io/en/latest/api_docs.html#module-gamornet.tflearn_module).

In [ ]:
from gamornet.tflearn_module import gamornet_train_tflearn

model = gamornet_train_tflearn(training_imgs,training_labels,valdiation_imgs,
                             validation_labels,input_shape='SDSS', epochs=20, 
                             max_checkpoints=2, batch_size=64, lr=0.0001, 
                             loss='categorical_crossentropy',clear_session=True)

The above command trains a model using the images we prepared for 20 epochs (or defined periods of time) using a learning rate of 0.0001 and a categorical cross-entropy loss function. The `max_checkpoints = 2` parameter ensures that the latest 2 snapshots of the epochs will always be saved during training. Three files are saved for each snapshot and the naming format of the checkpoints is `check-x.data`,`check-x.index`,`check-x.meta` where x refers to the step number at which the model was saved. The `input_shape` parameter specifies the shape of the input images. Setting this to `SDSS` automatically sets the value to `(167,167,1)`. 

In the output above, the `acc` and `loss` refer to the accuracy and loss calculated on the training set at the end of each epoch while `val_loss` and `val_acc` refer to the metrics calculated on the validation data. 

The `clear_session = True` parameter value instructs GaMorNet to clear the TensorFlow graphs created earlier. We highly recommend setting `clear_session` to `True` in notebooks while using the `tflearn_module` as otherwise it might fail. 

For an explanation of the different input parameters of `gamornet_train_tflearn`, please have a look at the [API documentation](https://gamornet.readthedocs.io/en/latest/api_docs.html).

**Tip:** Unlike with the keras module, the tflearn module doesn't automatically save the metrics. Instead you have to redirect the Python output generated to a file in order to keep track of the metrics. 

When running some python script this can be done simply using `python script.py > out.txt`. This will save all the screen output in `out.txt`.

Thereafter the following snippet of Python Code can easily search for the relevant metrics in the screen output file. 

```python
###################################
# accParser.py
#
# Takes tflearn screen output and extracts loss, acc and val_acc every epoch for visualization
####################################
import sys

if (len(sys.argv) != 2):
        print "Exiting Program....\nUsage: python accParser.py /path/to/screen/output"


dataPath = sys.argv[1] #the first argument is the path to the screen grab of the TF Learn run

dataFile = open(dataPath, 'r')
outFile = open(dataPath[:-6] + 'out.txt', 'w')

outFile.write("epoch loss acc val_acc\n")
resultLines = dataFile.readlines()

for line in resultLines:
        if 'val_acc' in line:
                words = line.split()

                #validation step
                if words[-2:-1] != ['iter:']:
                        print "Something doesn't look right. Skipping an occurene of val_acc"
                        continue

                outFile.write(words[words.index("epoch:")+1] + " ")
                outFile.write(words[words.index("loss:")+1] + " ")
                outFile.write(words[words.index("acc:")+1] + " ")
                outFile.write(words[words.index("val_acc:")+1] + "\n")

dataFile.close()
outFile.close()

```

**Important:** We highly recommend checking how the loss and accuracies vary with training. This is extremely helpful in judging whether the model was trained properly and sufficiently. 

### Summary & Takeaways

* `gamornet_train_keras` and `gamornet_train_tflearn` are the two functions that can be used to train GaMorNet models. 

* For understanding the differences between the Keras and TFLearn modules, please refer to the [PDR Handbook](https://gamornet.readthedocs.io/en/latest/usage_guide.html). 

* The [PDR Handbook](https://gamornet.readthedocs.io/en/latest/usage_guide.html) also contains advice about which situations warrant the training of models from scratch and in which cases you can use the models which we have released. 

# Making Morphological Classification Predictions via `GaMorNet`

Let's now used our trained neural network to start classifying galaxies in SDSS. For the purposes of this demonstration, we are going to use the models trained on SDSS simulations and real data in the paper. We are going to perform the prediction on two SDSS g-band images from our testing dataset. 

## Downloading & Visualizing the Data

First, let's download the images that we are going to use to perform the predictive analysis. We will download these into the local filesystem from Yale Astronomy's FTP service, where these are hosted.


*Tip: The `%%bash` command lets Colab know that all the commands in this shell needs to be passed the local unix virtual environment.*

In [ ]:
%%bash
#get zip file from server
wget ftp://ftp.astro.yale.edu/pub/aghosh/gamornet_tutorial_files/predict_images/predict_images.tar.gz

#create new directory and unzip to it
mkdir predict_images
tar -xvf predict_images.tar.gz -C ./predict_images/

Now, let's take a quick look at these two images.

In [ ]:
import matplotlib as mpl
plt.figure(dpi=120)
plt.subplot(1,2,1) #setting up 1st subplot

#reading in the data and plotting the first image
img_1_data = fits.getdata("./predict_images/587722984439545906-g.fits")
plt.imshow(img_1_data)

plt.subplot(1,2,2) #setting up the 2nd subplot

#reading in the data and plotting the second image
img_2_data = fits.getdata("./predict_images/587725552281976904-g.fits")
plt.imshow(img_2_data)

## Performing Predictions with `Keras`

Now, we will be using the models trained on SDSS simulations and real data from the paper in order to perform predictions on the two above images. 

In [ ]:
from gamornet.keras_module import gamornet_predict_keras

#Adding an explicit third axis to the image data (as this is needed for
#the GaMorNet function to work properly). Just to be clear the images
#were already 167*167, so we are not resizing the X,Y dimensions in any
#way
img_1_data = np.reshape(img_1_data,newshape=(167,167,1))
img_2_data = np.reshape(img_2_data,newshape=(167,167,1))
img_array = np.array([img_1_data,img_2_data])

#Performing Predictions
predictions = gamornet_predict_keras(img_array, model_load_path='SDSS_tl', input_shape='SDSS', batch_size=64)

print("COMPLETE!")

The `model_load_path = 'SDSS_tl'` is a special parameter value which automatically downloads and uses the final SDSS model of the paper. The `input_shape` parameter specifies the shape of the input images. Setting this to `SDSS` automatically sets the value to `(167,167,1)`

 For an explanation of each of the arguments of `gamornet_predict_keras`, please have a look at the [API Documentation](https://gamornet.readthedocs.io/en/latest/api_docs.html)

## Understanding the Predictions Made By `Keras`

In [ ]:
predictions.shape

As the above command shows, the returned predictions array is of the shape (# of images, 3) where for each image we have one single-dimensional array with 3 elements where

* Element 0 is the probability for the image to be *disk-dominated*
* Element 1 is the probability for the image to be *indeterminate*
* Element 2 is the probability for the image to be *bulge-dominated*

In [ ]:
#1st image predictions 
print(predictions[0])

#2nd image predictions
print(predictions[1])

As can be seen from the above arrays, GaMorNet predicts with 99.6% confidence that the first image is a disk-dominated galaxy and with 99.37% confidence that the second image is a bulge-dominated galaxy. 

These are indeed correct predictions as can be verified by investigating the light profiles of the galaxies in detail. 

*Tip: The number in the name of each image file refers to their SDSS Object ID*

***

#### Q - Reflect on the predictions you made earlier regarding applications for machine learning in astronomy. Where and why might it be advantageous to use neural networks to process astronomical data? Why, in particular, might it be useful in the context of galaxy studies?

***

Ans. Here

That's it for today's final tutorial on machine learning! Congrats on finishing up Part 5 and making it all the way through to the end of our Python in Astronomy workshop! 🎉